In [2]:
# Import packages
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable
import pandas as pd
import math
import csv
import numpy as np

# Load data
full_df = pd.read_csv('../../../Data/SMTO_2019/SMTO_2019_Complete_Input.csv')
school_codes = full_df['School'].unique().tolist()
uni_codes = full_df[full_df['School_Type'] == 'University']['School'].unique().tolist()

# Convert School column to numeric
full_df['School'] = full_df['School'].apply(lambda x: school_codes.index(x))

# Remove rows with missing information
full_df = full_df.dropna(subset = ['Family'])
full_df['Family'] = (full_df['Family'] * 1).astype(int)

In [10]:
# Load enrollment data
enrol_df = pd.read_csv('../../../Data/School_Info_2019_Pred_Enrol.csv').set_index('Code')

def code_to_log_enrol(code):
    """
    Return natural logarithm of total enrollment of campus with given code
    If code is invalid, raise KeyError
    If no enrollment information available for that code, return np.nan
    """
    return math.log(enrol_df.loc[code]['Total'])

In [11]:
def get_accuracy(cm):
    """
    Given confusion matrix as 2D array, return accuracy
    """
    correct = sum([cm[i][i] for i in range(len(cm))])
    return correct/sum(sum(cm,[])) * 100

In [22]:
cols_to_keep = ['School', 'Family'] + ['Dist.' + code for code in school_codes]
database = db.Database("SMTO_2019", full_df[cols_to_keep])
unis, cols = [], []
V, av = {}, {}
B_DIST = Beta('B_DIST', 0, None, None, 0)
B_FAM_DIST = Beta('B_FAM_DIST', 0, None, None, 0)
UNI = Beta('UNI', 3, 1, 10, 0)
COL = Beta('COL', 3, 1, 10, 0)

for i in range(len(school_codes)):
    code = school_codes[i]
    if code in uni_codes:
        unis.append(i)
    else:
        cols.append(i)
    V[i] = code_to_log_enrol(code) + database.variables['Dist.' + code] * (B_DIST + B_FAM_DIST * database.variables['Family'])
    av[i] = 1

UNIS = (UNI, unis)
COLS = (COL, cols)
nests = UNIS, COLS

In [23]:
import biogeme.messaging as msg
logprob = models.lognested(V, av, nests, database.variables["School"])
logger = msg.bioMessage()
logger.setGeneral()

biogeme = bio.BIOGEME(database, logprob)
results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()

[15:44:59] < General >   Remove 0 unused variables from the database as only 29 are used.
[15:45:00] < General >   Log likelihood (N=11005):  -35223.62
[15:45:00] < General >   Minimize with tol 1e-07
[15:45:06] < General >   Log likelihood (N=11005):  -35223.62 Gradient norm:      4e+05  
[15:45:12] < General >   Log likelihood (N=11005):  -867325.4 Gradient norm:      9e+05  
[15:45:18] < General >   Log likelihood (N=11005):  -78074.59 Gradient norm:      7e+05  
[15:45:23] < General >   Log likelihood (N=11005):  -36504.69 Gradient norm:      3e+05  
[15:45:29] < General >   Log likelihood (N=11005):  -34784.08 Gradient norm:      5e+04  
[15:45:35] < General >   Log likelihood (N=11005):  -34768.94 Gradient norm:      3e+04  
[15:45:41] < General >   Log likelihood (N=11005):   -34753.8 Gradient norm:      2e+04  
[15:45:48] < General >   Log likelihood (N=11005):  -34746.93 Gradient norm:      2e+04  
[15:45:54] < General >   Log likelihood (N=11005):   -34738.5 Gradient norm:   

In [24]:
pandasResults

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
B_DIST,-0.016306,0.0,0.000572,-28.483645,0.000000,0.001134,-14.373134,0.000000
B_FAM_DIST,0.001720,0.0,0.000715,2.404727,0.016185,0.001353,1.271577,0.203524
COL,1.230721,0.0,0.022219,55.390658,0.000000,0.028227,43.600902,0.000000
UNI,1.000000,1.0,0.012706,78.701754,0.000000,0.014752,67.788633,0.000000


In [27]:
betas = results.getBetaValues()    
print(betas)

simulate = {'Prob.' + school_codes[i]: models.logit(V, av, i) for i in range(len(school_codes))}
sim_biogeme = bio.BIOGEME(database, simulate)
probs = sim_biogeme.simulate(betas).set_index(full_df.index)

probs

{'B_DIST': -0.016305694241067592, 'B_FAM_DIST': 0.0017203764855639547, 'COL': 1.2307211387911412, 'UNI': 1.0}
[15:58:41] < General >   Remove 1 unused variables from the database as only 28 are used.


,Prob.CPR,Prob.CMO,Prob.CAS,Prob.CST,Prob.CDV,Prob.CEG,Prob.CDS,Prob.CPI,Prob.DOS,Prob.DWH,...,Prob.OTD,Prob.RY,Prob.SHT,Prob.SHD,Prob.SHH,Prob.SG,Prob.SC,Prob.MI,Prob.YK,Prob.YG
0,0.040572,0.013450,0.009005,0.006149,0.000177,0.000082,0.000093,0.000079,0.020781,0.004317,...,0.016937,0.129675,0.043418,0.042279,0.029925,0.250090,0.039507,0.075322,0.170196,0.009672
1,0.051669,0.016836,0.010798,0.006925,0.000224,0.000102,0.000100,0.000098,0.025392,0.005359,...,0.020611,0.138755,0.019943,0.029917,0.013130,0.270875,0.048869,0.054499,0.205969,0.012469
2,0.067214,0.024248,0.012217,0.006700,0.000181,0.000086,0.000093,0.000192,0.050177,0.010421,...,0.041781,0.124055,0.016352,0.023601,0.011142,0.235260,0.070860,0.044092,0.167131,0.011752
4,0.068652,0.022332,0.012560,0.006849,0.000205,0.000091,0.000094,0.000145,0.034812,0.007980,...,0.027838,0.126391,0.017222,0.027499,0.011338,0.239691,0.064480,0.047183,0.202589,0.012634
5,0.064122,0.021194,0.013592,0.007709,0.000182,0.000094,0.000109,0.000121,0.031207,0.006428,...,0.025252,0.145030,0.017626,0.024114,0.012237,0.274570,0.062306,0.049706,0.165938,0.012677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16912,0.070442,0.023097,0.012572,0.006856,0.000203,0.000090,0.000095,0.000144,0.035565,0.007913,...,0.028615,0.126660,0.017096,0.026643,0.011370,0.240200,0.066688,0.046839,0.196294,0.012628
16914,0.048970,0.015795,0.009931,0.006404,0.000245,0.000095,0.000093,0.000097,0.023169,0.005205,...,0.018480,0.129501,0.019616,0.032557,0.013098,0.254569,0.045551,0.057154,0.247121,0.011797
16915,0.058355,0.019040,0.011551,0.006745,0.000211,0.000092,0.000092,0.000146,0.034651,0.008859,...,0.027462,0.126144,0.017817,0.032140,0.011730,0.241063,0.054973,0.049384,0.216401,0.012564
16916,0.052723,0.017143,0.011573,0.007401,0.000219,0.000108,0.000105,0.000096,0.024781,0.005104,...,0.020052,0.145492,0.018425,0.028261,0.012303,0.278384,0.049733,0.052820,0.200052,0.013531


In [31]:
def get_cm(probs, hardmax):
    cm = []
    if hardmax:
        for school in range(len(school_codes)):
            cm.append([(probs[full_df['School'] == school][['Prob.' + i for i in school_codes]].idxmax(axis = 1) == 'Prob.' + j).sum() for j in school_codes])
    else:
        for school in range(len(school_codes)):
            cm.append((probs[full_df['School'] == school][['Prob.' + i for i in school_codes]].sum().values.tolist()))    
    return cm

def get_accuracy(cm):
    correct = sum([cm[i][i] for i in range(len(school_codes))])
    return correct/sum(sum(cm,[])) * 100

In [32]:
hard_cm = get_cm(probs, True)
soft_cm = get_cm(probs, False)
print("Hardmax Accuracy: {:2.2f} %".format(get_accuracy(hard_cm)))
print("Softmax Accuracy: {:2.2f} %".format(get_accuracy(soft_cm)))
print()

Hardmax Accuracy: 24.51 %
Softmax Accuracy: 14.35 %

